In [1]:
#Solo descomenta esta celda si quieres tu iniciar sesión personal en Google Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install osmnx rasterio mapclassify contextily

# Uso de rasterio y reclasificación de datos

## Primer ejemplo

In [4]:
import os
import zipfile
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from shapely.geometry import box
import warnings
warnings.filterwarnings("ignore")

In [5]:
ruta = os.path.join('drive', 'MyDrive', 'Verano', 'ArcGIS', 'Curso_Python_SIG', 'datos')
ruta_extraccion = os.path.join(ruta, 'extracciones')

with zipfile.ZipFile(os.path.join(ruta, 'uso_de_suelo.zip'), 'r') as zip_ref:
    zip_ref.extractall(ruta_extraccion)

En este ejemplo, utilizaremos una capa de uso de suelo y la reclasificaremos para crear una nueva capa que represente diferentes categorías de uso de suelo.

*  **`El uso de suelo`**  se refiere a la clasificación y regulación de las actividades permitidas en un área geográfica específica. En esencia, determina qué tipo de construcciones o actividades pueden realizarse en un terreno o inmueble. Esta regulación es establecida por los instrumentos de planificación territorial y urbanística, como los Planes de Desarrollo Urbano, que definen los usos permitidos en cada zona.
En términos más sencillos, el uso de suelo indica si un terreno puede ser utilizado para:
Vivienda (Habitacional): Construcción de casas, edificios de departamentos, etc.
Comercio: Tiendas, restaurantes, oficinas, etc.
Industria: Fábricas, almacenes, etc.
Servicios: Escuelas, hospitales, parques, etc.
Combinaciones de los anteriores: Algunas zonas pueden permitir usos mixtos, como comercio y vivienda.

![](https://github.com/patymunoz/curso-geoespacial/blob/main/source/images/usosuelo-gdl.png?raw=1)

In [7]:
ruta_df = os.path.join(ruta_extraccion, 'f1312_usv250s5d.shp')
gdf = gpd.read_file(ruta_df)

Debemos asegurarnos de que el CRS de la capa esté en un sistema que sea compatible para interpretar ``_metros_`` como unidades, porque vamos a realizar operaciones para calcular áreas:

Generamos una nueva columna para guardar los resultados de la reclasificación:

## Segundo ejemplo

### (A)

#### Obtener imagen del satélite _Shuttle Radar Topography Mission (SRTM)_

``USGS/SRTMGL1_003`` es un conjunto de datos que contiene información de **elevación global**, con una resolución aproximada de 30 metros.

### (B)

#### _OpenStreetMap_

Introducimos también la biblioteca de `osmnx` para descargar datos sobre: **carreteras, caminos y parques**.

In [ ]:
import osmnx as ox

In [ ]:
# Código segundo

## Rasterio

In [ ]:
import rasterio

#### _Shuttle Radar Topography Mission (SRTM)_

[SRTM](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-shuttle-radar-topography-mission-srtm-1)

* Cada píxel de la imagen representa una altura en metros sobre el nivel del mar.

* Es un raster georreferenciado, lo que significa que cada píxel tiene una ubicación exacta en la Tierra.

* Resolución: 30 metros (cada píxel representa un cuadrado de 30x30 metros en el terreno).

Sombras en escala de grises: representan variación de altitud

* Negro / oscuro: zonas más bajas

* Blanco / claro: zonas elevadas (cerros, sierras)

![](https://github.com/patymunoz/curso-geoespacial/blob/main/source/images/elev.png?raw=1)

Crear una **rejilla de celdas cuadradas** sobre una región geográfica específica, con cada celda de aproximadamente 500 metros de lado:

In [ ]:
# Bounding box
xmin, ymin, xmax, ymax = -103.6, 20.55, -103.2, 20.75
cell_size = 0.005  # ~500 metros (aprox)

cols = np.arange(xmin, xmax, cell_size)
rows = np.arange(ymin, ymax, cell_size)

polygons = []
for x in cols:
    for y in rows:
        poly = box(x, y, x + cell_size, y + cell_size)
        polygons.append(poly)

grid = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")

Vamos a "recortar" el área de interés entre `grid` y los objetos vectoriales `road` y `parks`

In [ ]:
# Código tercero

### Generamos los cálculos

Este bloque de código extrae valores de elevación desde un raster (``elev_jalisco.tif``) para cada celda de la cuadrícula, usando el _centroide_ de cada celda como punto de muestreo.

Pasos clave:

* Se abre el raster de elevación.

* Se calculan los centroides de cada celda del grid y se reproyectan al sistema de coordenadas del raster.

* Se extraen los valores de elevación en esos puntos.

* Se añade una columna ``'elev'`` al GeoDataFrame con los valores obtenidos.

In [ ]:
with rasterio.open(os.path.join('drive', 'MyDrive', 'tu carpeta del curso', 'datos','elev_jalisco.tif')) as src:
    centroids_proj = grid.set_geometry('centroid').to_crs(src.crs)
    coords = [(pt.x, pt.y) for pt in centroids_proj.geometry]
    elev_values = [val[0] for val in src.sample(coords)]

grid['elev'] = elev_values

Este código reproyecta todos los datos al sistema ``UTM zona 13N (EPSG:32613)``, lo cual permite medir distancias en metros reales. Luego calcula la distancia desde el centroide de cada celda del grid a la carretera y parque más cercanos.

In [ ]:
# Reproyectar todo al mismo CRS proyectado
grid_proj = grid.to_crs(epsg=32613)
roads_proj = roads.to_crs(epsg=32613)
parks_proj = parks.to_crs(epsg=32613)

# Recalcular centroides después de reproyección
grid_proj['centroid'] = grid_proj.geometry.centroid

# Calcular distancias en metros
grid_proj['dist_road'] = grid_proj.centroid.apply(lambda pt: roads_proj.distance(pt).min())
grid_proj['dist_park'] = grid_proj.centroid.apply(lambda pt: parks_proj.distance(pt).min())

- Se usa **`EPSG:32613`** (UTM zona 13N) para garantizar que las unidades estén en metros.
- Se usa `.distance(pt).min()` para obtener la **distancia mínima** entre un punto y todos los objetos de referencia (carreteras o parques).
- El resultado son dos nuevas columnas: `dist_road` y `dist_park`, con las distancias en metros.

Creamos 3 nuevas columnas con datos numéricos (``elev``, ``dist_road``, ``dist_park``) que representan:

### Clasificación de mapas temáticos

In [ ]:
import mapclassify

### Reclasificación del área de interés

Ahora, construyamos una función que nos permita reclasificar las tres características de interés:

#### Criterios para clasificar un área de interés:

- Altura menor a **1700 metros** (`elev < 1700`)
- Estar a más de **900 metros** de una carretera (`dist_road > 900`)
- Estar a menos de **1000 metros** de un parque (`dist_park < 1000`)

In [ ]:
# Código cuarto

In [ ]:
import contextily as cx